In [1]:
import stata_setup
import pandas as pd
stata_setup.config("/Applications/STATA","se")


  ___  ____  ____  ____  ____ ®
 /__    /   ____/   /   ____/      18.0
___/   /   /___/   /   /___/       SE—Standard Edition

 Statistics and Data Science       Copyright 1985-2023 StataCorp LLC
                                   StataCorp
                                   4905 Lakeway Drive
                                   College Station, Texas 77845 USA
                                   800-STATA-PC        https://www.stata.com
                                   979-696-4600        stata@stata.com

Stata license: Unlimited-user network, expiring 31 Aug 2024
Serial number: 401809300159
  Licensed to: Bruno Komel
               University of Pittsburgh

Notes:
      1. Unicode is supported; see help unicode_advice.
      2. Maximum number of variables is set to 5,000 but can be increased;
          see help set_maxvar.


In [ ]:
%%stata

global recitation "~/Documents/Pitt/Year_2/TA - Econ 3080/Recitations/Recitation 8 - Fuzzy RDD"
cd "${recitation}"

// Thanks to https://evalf20.classes.andrewheiss.com/example/rdd-fuzzy/#fuzzy-parametric-estimation for this
import delimited "tutoring_program_fuzzy.csv", clear 

**Here's the setting:
* Students take an entrance exam at the beginning of the school year
* If they score 70 or below, they are enrolled in a free tutoring program
* BUT we have some non-compliers. Both always takers and never takers.
* Students take an exit exam at the end of the year

In [ ]:
%%stata

// So, let's take a look at what we have:

xi: twoway scatter i.tutoring_text entrance_exam, xline(70, lcolor(red)) 
    
// The first thing that we can notice is that the cutoff is not strict. 
//There are observations on both sides of the cutoff that fall in either category.

In [ ]:
%%stata

// Next, we should try to get an idea of how many people are in each group (compliers, non-compliers, etc)
// Let's generate a dummy for the "treatment," which would have been assigned if we had a sharp RDD
gen below_cutoff = 0
replace below_cutoff = 1 if entrance_exam <= 70

tab tutoring, gen(tutoring)

bysort below_cutoff tutoring2: sum tutoring2

In [ ]:
%%stata

// Let's see if we can find some discontinuity in the plots?
twoway (scatter exit_exam entrance_exam if tutoring2 == 1 ) ///
	(scatter exit_exam entrance_exam if tutoring2 == 0) ///
	(lfitci  exit_exam entrance_exam if tutoring2 == 1& entrance_exam <= 70 , ciplot(rline) color(red)) ///
	(lfitci  exit_exam entrance_exam if tutoring2 == 1& entrance_exam > 70 , ciplot(rline) color(red)) ///
	(lfitci  exit_exam entrance_exam if tutoring2 == 0& entrance_exam <= 70 ,  ciplot(rarea) color(green)) ///
	(lfitci  exit_exam entrance_exam if tutoring2 == 0& entrance_exam > 70 , ciplot(rarea) color(green)), /// 
	legend(lab(1 "With Tutoring") lab(2 "No Tutoring") lab(4 "Tutoring Below Fit.") ///
	lab(6 "Tutoring Above Fit.") lab(8 "No Tut. Below Fit.") lab(10 "No Tut. Above Fit."))

In [ ]:
%%stata


// Next, just so we can get an idea of how many compliers, and non-compliers we have, let's plot this
// differently. Here I want to see the probability of receiving tutoring for the different socres that 
// people received, using bins of 5 points each.

// To use the twoway bar command, we need to create a categorical variable for each score bin
gen entr_score_cat =0

forvalues i = 25(5)95{
	replace entr_score_cat = `i'/5 - 5 if entrance_exam > `i' & entrance_exam <= `i'+5
}


In [ ]:
%%stata

// And add a label so it looks slightly better
forvalues i = 25(5)95{
	label define entr_score_cat_lab `=`i'/5 - 5' "`=`i'+1' - `=`i'+5'" , add
}


// I want to highlight what we're doing here, because it's not trivial.
// Using the index `i' inside the loop tells stata to use whatever value of `i' is currently "running" 
// in the loop.
// But what if you want a lagged index value? Or one with a lead?
// Then you need to use `= `i' + 1' . Where you're basically running a small equation inside the ` ' 

In [ ]:
%%stata

label values entr_score_cat entr_score_cat_lab

save recitation8_data, replace

In [ ]:
%%stata -doutd df2

use recitation8_data, replace

In [ ]:
display(df2)
# Note that in Stata, this would the "entr_score_cat" would display as the labels we attached

In [ ]:
%%stata

// And to get the probability, we need to generate a "frequency" by taking the mean of the dummy variable 
// over the number of observarions in each category

egen frequency = mean(tutoring2), by(entr_score_cat)

In [ ]:
%%stata

graph bar tutoring2, over(entr_score_cat, gap(100)) outergap(*1.2) ytitle("Probability of Receiving Tutoring")

In [ ]:
%%stata


twoway (bar frequency  entr_score_cat,  xline(8.5, lwidth(thick) lcolor(red)) ///
        xlabel(0(1)14,valuelabel alternate) barwidth(0.8)  ytitle("Probability of Receiving Tutoring") ///
        xtitle("Entrance Exam Score") )

In [ ]:
%%stata

// The key here is that some students above the cut-off receive tutoring, and some below the cut-off do not

// Next, let's recenter our running variable
gen entrance_centered = entrance_exam - 70

// recall we've generated below_cutoff

In [ ]:
%%stata


// Let's pretend this is a sharp RDD scenario:
// first let's label some stuff
label variable exit_exam "Exit Exam"
label variable tutoring2 "Received Tutoring"

In [ ]:
%%stata


// And let's store our regressions to create a nice table 
eststo OLS: reg exit_exam entrance_centered tutoring2 if (entrance_centered >= -10 & entrance_centered <=10)
    estadd scalar BW = 10
// In this case we'd estimate an effect of 11.48
// Note that I want to keep track of the size of the bandwidth for each of these regressions

In [ ]:
%%stata
// ssc install ivreg2
// ssc install ranktest

eststo IV: ivreg2 exit_exam entrance_centered ( tutoring2 = below_cutoff ) ///
    if (entrance_centered >= -10 & entrance_centered <=10), robust 
    estadd scalar BW = 10
// Now the coefficeint dropped to 9.74. This is the effect on compliers in the bandwidth (which we chose)

In [ ]:
%%stata

eststo RDrob: rdrobust exit_exam entrance_centered, fuzzy(tutoring2) c(0)
    estadd scalar BW = e(h_r)
// And here it's pretty close to the 2sls
// Note that you do not need to specify an insrtument with rdrobust.

In [ ]:
%%stata

global latex "/Users/brunokomel/Library/CloudStorage/Dropbox/Apps/Overleaf/Recitation - Tables"
cd "${latex}"

// Using this method we get pretty close results!
esttab OLS IV RDrob using "table1.tex",  sfmt(4) b(3) se(2) keep(tutoring2 RD_Estimate) ///
varlabel(RD_Estimate "Received Tutoring (Non-Parametric)") label mtitles("OLS" "2SLS" "RD Robust") ///
scalars("N Observations" "BW Bandwidth Choice") fragment replace

/// Here, esttab will tell stata to compile your table
/// using creates, or overwrites, a .tex file where your table will go (it'll go to the working directory)
/// sfmt(x) will tell stata to show x characters after the decimal point for the "general table values" (catch-all)
/// b(x) will tell stata to show x characters after the decimal point for the BETA coefficients
/// se(x) will tell stata to show x characters after the decimal point for the STANDARD ERROS
/// varlabel(z " Z " ) tells stata to label, on the table, the variable z as " Z "
/// label tells stata to use the labels of the variables that have labels
/// mtitles("Col 1 " "Col 2" ...) assigns column titles to each of the estimates that you stored and appended
/// scalars("X Name of X you want" ...) tells stata to output the stored scalar/local "X" and give it the name that you want 
/// fragment tells stata that there will be more esttab's coming to append a table, and it removes some automatic things (like star captions)                                                                        
/// replace will automatically save over the table that you previously created

In [ ]:
%%stata 

esttab OLS IV RDrob ,  sfmt(4) b(3) se(2) keep(tutoring2 RD_Estimate) ///
varlabel(RD_Estimate "Received Tutoring (Non-Parametric)") label mtitles("OLS" "2SLS" "RD Robust") ///
scalars("N Observations" "BW Bandwidth Choice") fragment 

# Exercise

In [ ]:
%%stata

cd "${recitation}"

use analysis.dta, clear

// From Vincent Ponz and Clemente Tricaud's papper 
// "Expressive Voting and its Cost: Evidence from Runoffs With Two or Three Candidates"
// Econometrica 2018
//https://www.econometricsociety.org/publications/econometrica/2018/09/01/expressive-voting-and-its-cost-evidence-runoffs-two-or-three

In [ ]:
%%stata

use analysis.dta, clear

In [ ]:
%%stata

keep prop_registered_turnout_R2 running treatment assignment prop_registered_blanknull_R2 ///
prop_registered_candvotes_R2 year prop_registered_votes_cand3_R1

rename prop_registered_votes_cand3_R1 running_uncentered
rename prop_registered_turnout_R2 turnout
rename prop_registered_blanknull_R2 blanknull
rename prop_registered_candvotes_R2 candvotes

In [ ]:
%%stata

// Let's plot some stuff so we get an idea of what's happening
// First, the "first stage"
rdplot treatment running, p(1) graph_options(title("") ///
    ytitle(Treatment status) xtitle(Running variable) graphregion(color(white)) ///
    legend(off) ylabel(0 (.2) 1) xlabel(-.1 (.05) .1))) nbins(20 20)
// Note how we don't have perfect compliance. Not all candidates who get more than
// 12.5% end up running in the second round

In [ ]:
%%stata

// Second, let's look at one of the outcomes 
rdplot turnout running,  fuzzy(treatment) nbins(30 30) p(2). graph_options(title("") legend(off) ///
    ytitle(Candidate votes 2nd round) xtitle(Running variable) graphregion(color(white)) ylabel(.2(.2) 1) ///
    xlabel(-.15 (.05) .15))) 
// They're using nbins to define the bin width 

In [ ]:
%%stata

// We can also add confidence intervals:
rdplot turnout running,  fuzzy(treatment) nbins(30 30) p(2). graph_options(title("") legend(off) ///
    ytitle(Candidate votes 2nd round) xtitle(Running variable) graphregion(color(white)) ylabel(.2(.2) 1) ///
    xlabel(-.15 (.05) .15))) ci(95) shade 

In [ ]:
%%stata 

rdplot turnout running,  nbins(30 30) p(2). graph_options(title("") legend(off) ///
    ytitle(Candidate votes 2nd round) xtitle(Running variable) graphregion(color(white)) ylabel(.2(.2) 1) ///
    xlabel(-.15 (.05) .15)))
// Here I just want to show that including the "fuzzy" option doesn't change anything here

ereturn list


In [ ]:
%%stata

// Replicate and generate table 3, then add a panel below using the 2sls estimates
eststo rdrob_turn: rdrobust turnout running, fuzzy(treatment) 
    estadd scalar obs=e(N_h_l)+e(N_h_r)
    estadd scalar pval = e(pv_rb)
    estadd scalar poly = e(p)
    estadd scalar BW = e(h_r)
    estadd local bws = e(bwselect)
sum turnout if assignment==0 & abs(running)<= e(h_r)
local mean_control=r(mean)
    estadd scalar mean_con = `mean_control'

In [ ]:
%%stata

eststo rdrob_blank: rdrobust blanknull running, fuzzy(treatment) 
    estadd scalar obs=e(N_h_l)+e(N_h_r)
    estadd scalar pval = e(pv_rb)
    estadd scalar poly = e(p)
    estadd scalar BW = e(h_r)
    estadd local bws = e(bwselect)
sum blanknull if assignment==0 & abs(running)<= e(h_r)
local mean_control=r(mean)
    estadd scalar mean_con = `mean_control'

In [ ]:
%%stata

eststo rdrob_cand: rdrobust candvotes  running, fuzzy(treatment) 
    estadd scalar mean_control=r(mean)
    estadd scalar obs=e(N_h_l)+e(N_h_r)
    estadd scalar pval = e(pv_rb)
    estadd scalar poly = e(p)
    estadd scalar BW = e(h_r)
    estadd local bws = e(bwselect)
sum candvotes if assignment==0 & abs(running)<= e(h_r)
local mean_control=r(mean)
    estadd scalar mean_con = `mean_control'

In [ ]:
%%stata

eststo rdrob_cand_yr: xi: rdrobust candvotes  running, fuzzy(treatment) covs(i.year)

In [ ]:
%%stata

ereturn list

In [ ]:
%%stata

// Now I want to show you how to account for covariates in the rdrobust command	
eststo rdrob_cand_yr: xi: rdrobust candvotes  running, fuzzy(treatment) covs(i.year)
    estadd scalar mean_control=r(mean)
    estadd scalar obs=e(N_h_l)+e(N_h_r)
    estadd scalar pval = e(pv_rb)
    estadd scalar poly = e(p)
    estadd scalar BW = e(h_r)
    estadd local bws = e(bwselect)
sum candvotes if assignment==0 & abs(running)<= e(h_r)
local mean_control=r(mean)
    estadd scalar mean_con = `mean_control'

In [ ]:
%%stata

cd "${latex}"

esttab rdrob_turn rdrob_blank rdrob_cand rdrob_cand_yr using "table2.tex",  sfmt(4) b(3) se(2) ///
varlabel(RD_Estimate "3rd. Present") ///
scalars("pval Robust p-value" "obs Observations" "poly Polyn. Order" "BW Bandwidth" "bws Band. method" "mean_con Mean, left of the threshold") ///
label mtitles("Turnout" "Null and Blank Votes" "Candidate Votes") noobs compress ///
posthead("\hline \\ \multicolumn{2}{c}{\textbf{Panel A: RD Robust Estimates}}\\\\ [-1ex]") ///
fragment /// This fragment will allow you to append a second panel below
replace

// fragment tells stata that there will be more esttab's coming to append a table
// noobs tells stata not to show the normal Observation count
// compress : I don't know exactly what it does, but the tables look bad without it
// posthead is some LaTex stuff that I don't really understand, but I copy and edit as I need it

In [ ]:
%%stata

// Note here that you will have to make a choice of bandwidth. I chose 0.02, because there were still enough observations on both sides to get significant estimates (and it's close to the optimal bandwidth above)
eststo iv_turn: ivreg2  turnout running (treatment  = assignment) if running >= -0.02 & running <= 0.02

eststo iv_blank: ivreg2  blanknull running (treatment  = assignment) ///
if running >= -0.02 & running <= 0.02

eststo iv_cand_year: ivreg2  candvotes running (treatment  = assignment) ///
if running >= -0.02 & running <= 0.02

eststo iv_cand_year2: xi: ivreg2 candvotes running (treatment  = assignment) ///
i.year if running >= -0.02 & running <= 0.02


In [ ]:
%%stata

esttab iv_* using "table2.tex", sfmt(4) b(3) se(2) keep(treatment) varlabel(treatment "3rd. Present") /// 
nomtitles booktabs nonumbers compress ///
posthead("\hline \\ \multicolumn{2}{c}{\textbf{Panel B: IV Estimates}}\\\\ [-1ex] ") ///
fragment ///
append 

// append here is the key that forces these estimates into the table we previously created, and called "using ..."
// nonumbers takes out the column numbers, which we don't need because we already have them in the table above
